In [ ]:
import pandas as pd
import numpy as np
from fastai import *
from fastai.tabular import *
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/kalapa data/trainV1.csv')
test_df = pd.read_csv('/content/drive/My Drive/kalapa data/testV1.csv')
label = train_df['label']

# Handle datetime features


In [ ]:
DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33,34, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1,2,43,44]]
date_need_correct = ['Field_34', 'ngaySinh']
start_end_date = [f'{cat}_startDate' for cat in ['F', 'E', 'C', 'G', 'A'] ] + [f'{cat}_endDate' for cat in ['F', 'E', 'C', 'G', 'A'] ]
date_names = DATE + DATETIME + start_end_date


**Convert string to date object**

In [ ]:
def str2date(s):
  if s!=s:
    return np.nan
  try:
    date_obj = datetime.strptime(str(s),'%m/%d/%Y')
  except:
    date_obj = datetime.strptime(str(s), '%Y-%m-%d')
  return date_obj
def correct_34_ngaysinh(s):
    if s != s:
        return np.nan
    try:
        s = int(s)
    except ValueError:
        s = s.split(" ")[0]
        
    return datetime.strptime(str(s)[:6], "%Y%m")
def datetime_normalize(s):
    if s != s:
        return np.nan
    
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime.date(datetime_obj)
def process_datetime(df):
  for col in DATE:
    if col == 'Field_34':
      continue
    df[col] = df[col].apply(str2date)
  df['Field_34'] = df['Field_34'].apply(correct_34_ngaysinh)
  df['ngaySinh'] = df['ngaySinh'].apply(correct_34_ngaysinh)
  for col in DATETIME:
    df[col] = df[col].apply(datetime_normalize)
  for col in start_end_date:
    df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
  return df
def make_new_features(df):
  for cat in ['F', 'E', 'C', 'G', 'A']:
    df[f'{cat}_span'] = df[f'{cat}_endDate'] - df[f'{cat}_startDate']
  def take_day(d):
    return d.days
  for cat in ['F', 'E', 'C', 'G', 'A']:
    df[f'{cat}_span'] = df[f'{cat}_span'].apply(take_day)
  return df


#processing datetime data in df
train_df = process_datetime(train_df)
test_df = process_datetime(test_df)
train_df = make_new_features(train_df)
test_df = make_new_features(test_df)




# Handle String features

In [ ]:
#process diaChi and maCv columns:
def str_normalize(s):
  s = str(s).strip().lower()
  s = re.sub(' +', " ", s)
  return s
def job_category(x):
  if type(x) == str:
    if "công nhân" in x or "cnv" in x or "cn" in x or "may công nghiệp" in x or "lao động" in x\
    or "thợ" in x or "coõng nhaõn trửùc tieỏp maựy may coõng nghieọp" in x or "c.n" in x or "lđ" in x:
        return "CN"
    elif "giáo viên" in x or "gv" in x or "gíao viên" in x:
        return "GV"
    elif "nhân viên" in x or "kế toán" in x or "cán bộ" in x or "nv" in x or "cb" in x or "nhõn viờn" in x:
        return "NV"
    elif "tài xế" in x or "lái" in x or "tài xê" in x:
        return "TX"
    elif "quản lý" in x or "phó phòng" in x or "hiệu phó" in x:
        return "QL"
    elif "undefined" in x:
        return "missing"
    elif "giám đốc" in x or "hiệu trưởng" in x:
        return "GĐ"
    elif "phục vụ" in x:
        return "PV"
    elif "chuyên viên" in x:
        return  "CV"
    elif "bác sĩ" in x or "dược sĩ" in x or "y sĩ" in x or "y sỹ" in x:
        return "BS"
    elif "y tá" in x:
        return "YT"
    elif "hộ sinh" in x:
        return "HS"
    elif "chủ tịch" in x:
        return "CT"
    elif "bếp" in x:
        return "ĐB"
    elif "sư" in x:
        return "KS"
    elif "dưỡng" in x:
        return "ĐD"
    elif "kỹ thuật" in x or "kĩ thuật" in x:
        return "KTV"
    elif "diễn viên" in x:
        return "DV"
    else:
        return "missing"
  else:
    return x    
  
def process_diaChi_maCv(df):
    df["maCv"] = df["maCv"].apply(str_normalize).apply(job_category).astype("category")
    return df
train_df = process_diaChi_maCv(train_df)
test_df = process_diaChi_maCv(test_df)

In [ ]:
#combine 2 gender features as 1
def combine_gender(s):
  x, y = s
  
  if x != x and y != y:
      return np.nan
  
  if x != x:
      return y.lower()
  
  return x.lower()

def process_gender(df):
  df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1).astype("category")
  df = df.drop(['gioiTinh', 'info_social_sex'],axis = 1)
  return df
train_df = process_gender(train_df)
test_df = process_gender(test_df)

In [ ]:
def process_misc(df):        
  df["subscriberCount"].replace(0, np.nan, inplace=True)
  df["friendCount"].replace(0, np.nan, inplace=True)
  
  df["Field_13"] = df["Field_13"].apply(lambda x: 1 if x == x else 0)
  df["Field_38"] = df["Field_38"].map({0: 0.0, 1: 1.0, "DN": np.nan, "TN": np.nan, "GD": np.nan})
  df["Field_62"] = df["Field_62"].map({"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5, "Ngoài quốc doanh Quận 7": np.nan})
  df["Field_47"] = df["Field_47"].map({"Zezo": 0, "One": 1, "Two": 2, "Three": 3, "Four": 4})
  
  df["Field_27"] = df["Field_27"].replace({0.0: np.nan})
  df["Field_28"] = df["Field_28"].replace({0.0: np.nan})
      
  for col in df.columns:
    if df[col].dtype.name == "object":
      df[col] = df[col].apply(str_normalize).astype("category")
          
  return df
train_df = process_misc(train_df)
test_df = process_misc(test_df)

In [ ]:
#Drop some field we do not need
# drop some fields we do not need (homeTown is optionally)
DROP = ["namSinh"] + \
        [f"Field_{c}" for c in [14, 16, 17,18, 24, 26, 30, 31, 37, 52, 57]]
train_df.drop(DROP, axis = 1,inplace = True)
test_df.drop(DROP, axis =1, inplace = True)



In [ ]:
#process diachi
def feature_diaChi(s):
    if type(s) == str:
        if 'hcm' in s or 'hồ chí minh' in s:
            return 'hcm'
        elif 'tỉnh' in s:
            return 'tinh'
        elif 'ha noi' in s or 'hà nội' in s:
            return 'hanoi'
        else:
            return 'tinh'
        
train_df['diaChi'] =train_df['diaChi'].apply(feature_diaChi)

In [ ]:
#process field_46
def field_46(s):
    if type(s) == str:
        
        if 'thất nghiệp' in s:
            return 'thatnghiep'
        elif s == 'nan':
            return s
        elif 'cty' in s or 'công ty' in s or 'c.ty' in s:
            return 'cty'
        elif 'đại lý' in s:
            return 'daily'
        elif 'đại học' in s or 'cđ' in s:
            return 'daihoc'
        elif 'hộ gia đình' in s or 'hgđ':
            return 'giadinh'
        else:
            return s
train_df['Field_46'] =  train_df['Field_46'].apply(field_46)

In [ ]:
#process Field_48
def Field_48(s):
    if type(s) == str:
        if 'hcm' in s or 'hồ chí minh' in s:
            return 'hcm'
        elif 'tỉnh' in s:
            return 'tinh'
        elif 'ha noi' in s or 'hà nội' in s:
            return 'hanoi'
        else:
            return s
        
train_df['Field_48'].apply(feature_diaChi).value_counts()

In [ ]:
dependent = 'label'


partner_features = [f'partner{i}_{S}' for i in range(0,6) for S in ['A','B','C','D','E','F','G','H','K','L'] ]

cat_names = ['Field_3', 'Field_4', 'Field_12', 'Field_13', 'gioiTinh', 'Field_36', 'Field_39', 'Field_41',
                                'Field_47', 'Field_54', 'Field_58', 'Field_61', 'Field_62', 'Field_65', 'Field_66', 'maCv', 'Field_73', 
                                'Field_76', 'Field_77', 'Field_78', 'gender']

num_names = ['Field_10', 'Field_19', 'Field_23', 'Field_27', 'Field_28', 'Field_29', 'Field_59', 'Field_60', 'Field_63', 'Field_64', 'Field_69', 'Field_70',
                         'Field_71', 'Field_72', 'Field_74', 'Field_75', 'friendCount', 'C_numOrg', 'C_numQuery', 'G_numOrg', 'G_numQuery', 'A_numOrg', 
                         'A_numQuery', 'summary_6m', 'summary_3m', 'summary_1m', 'summary_1w', 'Field_78', 'Field_79', 'Field_80', 'Field_81', 'Field_82','Field_20',
             'Field_20', ]


In [ ]:
train_df['Field_61'].astype(str)
train_df['Field_61']= train_df['Field_61'].replace(to_replace = ['Người sống tại vùng đặc biệt khó khăn', 'Đối tượng cận nghèo', 'Đối tượng nghèo','Người dân tộc thiểu số sống vùng KK, ĐBKK',
                                                'Khối DN Ngoài quốc doanh', 'Khối DN có vốn DTNN', 'Khối DN Nhà Nước', 'Doanh nghiệp LLVT','Trợ cấp bảo hiểm thất nghiệp',
                                               'Bảo trợ xã hội', 'Hưu trí, trợ cấp mất sức lao động', 'Hưởng chế độ thai sản','Hưu trí, trợ cấp mất sức lao động do NSNN đóng',
                                               'Ốm đau dài ngày','Hộ gia đình','Khối HS, Đảng, Đoàn','Học sinh sinh viên','Hộ gia đình làm nông lâm ngư diêm nghiệp',
                                               'Khối phường xã, thị trấn,','Khối ngoài công lập','Cán bộ phường xã không chuyên trách','Người sống tại xã đảo, huyện đảo',
                                               'Khối hợp tác xã','Hộ SXKD cá thể, tổ hợp tác','Thân nhân sĩ quan nghiệp vụ công an','Đại biểu quốc hội, HĐNN',
                                               'Thân nhân người có công','Tham gia kháng chiến','Cựu chiến binh','Trẻ em dưới 6 tuổi','Người đã hiến bộ phận cơ thể',
                                               'Người có công','Khối tạm dừng','TG','Tổ chức khác và cá nhân'], value = ['KK', 'KK', 'KK', 'KK',
                                                'DN', 'DN', 'DN','DN','MATVIEC', 'MATVIEC','MATVIEC','MATVIEC','MATVIEC','MATVIEC', 'KHAC','KHAC','KHAC','KHAC','KHAC','KHAC','KHAC',
                                                                           'KHAC','KHAC','KHAC','KHAC','KHAC','KHAC','KHAC','KHAC','KHAC','KHAC','KHAC','KHAC','KHAC','KHAC'])
#drop_field = ['Unnamed: 0','id', 'label','Field_18']
#train_df.drop(drop_field, inplace =True, axis = 1)

NameError: ignored

In [ ]:
ft = date_names + partner_features + cat_names + num_names
left_features = [feature  for feature in train_df.columns if not feature in ft]
print(left_features)

['Unnamed: 0', 'id', 'label', 'Field_18', 'Field_21', 'Field_22', 'ngaySinh', 'diaChi', 'Field_38', 'Field_42', 'Field_45', 'Field_46', 'Field_48', 'Field_49', 'Field_50', 'Field_51', 'Field_53', 'Field_55', 'Field_56', 'Field_67', 'Field_68', 'subscriberCount', 'currentLocationLocationId', 'currentLocationLatitude', 'currentLocationLongitude', 'homeTownLocationId', 'homeTownLatitude', 'homeTownLongitude', 'data.basic_info.locale', 'currentLocationCity', 'currentLocationCountry', 'currentLocationName', 'currentLocationState', 'homeTownCity', 'homeTownCountry', 'homeTownName', 'homeTownState', 'topFriends', 'numOrg', 'F_numOrg', 'F_numQuery', 'E_numOrg', 'E_numQuery', 'brief', 'num_of_phone']


#Create model

In [ ]:
label = label.astype('category')
procs = [FillMissing, Categorify, Normalize]
test = TabularList.from_df(test_df, cat_names= cat_names + date_names, cont_names= num_names, procs = procs)
data = (TabularList.from_df(train_df, cat_names=cat_names + date_names, cont_names=num_names, procs=procs)
                        .split_by_idx(list(range(0,5000)))
                        .label_from_df(cols = label)
                        .add_test(test, label=0)
                        .databunch())